In [1]:
import ges
import sempler
import numpy as np
import scipy.stats as st

## Find Causal Graph and get confidence interval [one trial]

In [231]:
d = 20 # of attributes
n = 500 # of datapoints

mu_lb, mu_ub = 0, 10 # range for means of the d components
sig_lb, sig_ub = 0, 10 # range for means of the variance components

In [232]:
# Generate observational data from a Gaussian SCM using sempler
G = np.zeros((d, d))

data = sempler.LGANM(G, (mu_lb, mu_ub), (sig_lb, sig_ub)).sample(n=n)

# Run GES with the Gaussian BIC score
estimate, score = ges.fit_bic(data, phases=['forward', 'backward'])

print(estimate, score)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] 1.7115809898222238


In [233]:
connections = np.where(estimate>0)

## Backdoor set

In [7]:
def get_parents(x, G):
    parents = []
    for i in range(G.shape[0]):
        if(G[i, x] == 1):
            parents.append(i)
    return parents

In [8]:
def get_all_family(x, G):
    visited = np.zeros(G.shape[0])
    visited[x] = 1
    
    x_parents = get_parents(x, G)
    to_search = x_parents
    reach_from_x = []
    
    while len(to_search):
        to_search_new = []
        
        for y in to_search:
            if(visited[y]):
                continue
            else:
                visited[y] = 1
                
            y_parents = get_parents(y, G)
            to_search_new += y_parents
            reach_from_x.append(y)
            
        to_search = to_search_new
        
    return reach_from_x

In [262]:
get_all_family(12, estimate)

[18]

In [263]:
get_all_family(18, estimate)

[12]

In [ ]:
intersection = [x for x in get_all_family(12, estimate) if x in b]

## Experiment Definition (assume n >= 30)

In [2]:
def get_conf_interval(a, b, conf_lvl=.95):
    effect_size, resid, _, _ = np.linalg.lstsq(a, b, rcond=None)
    sq_tot_dev = sum([(a_i - np.mean(a))**2 for a_i in a])
    SE = np.sqrt(resid / ((n-2) * sq_tot_dev))
    conf = st.norm.ppf(conf_lvl) * SE
    return (effect_size[0] - conf[0], effect_size[0] + conf[0])

In [5]:
def experiment(d=10, n=500, trials=100, mu_range=(0, 10), sig_range=(0,10)):
    success = 0
    for trial in range(trials):
        # start from empty causal graph, generate data & fit causal graph
        G = np.zeros((d, d))
        data = sempler.LGANM(G, mu_range, sig_range).sample(n=n)
        estimate, score = ges.fit_bic(data, phases=['forward', 'backward'])
        if(len(np.where(estimate>0)[0]) == 0): # GES found empty graph so it is correct and we stop early
            success += 1
            continue

        # o/w choose arbirary edge & find confidence interval of effect size
        connections = np.where(estimate>0)
        #idx = np.random.randint(0, len(connections[0]))
        
        for idx in range(len(connections)):
            ## check if needs backdoor adj
            backdoor = [x for x in get_all_family(connections[0][idx], estimate) \
                        if x in get_all_family(connections[1][idx], estimate)]
            if(len(backdoor) == 0):
                break

        A = data[:, connections[0][idx]].reshape((n,1))
        for node in backdoor:
            A = np.column_stack((A, data[:, node]))
        b = data[:, connections[1][idx]]
   
        (conf_lb, conf_ub) = get_conf_interval(A, b)

        # check if 0 is in the interval
        if(conf_lb <= 0 and 0 <= conf_ub):
            success+=1
    return success / trials

In [11]:
results = {}

for d in [15]:
    for n in range(100, 901, 200):
        results[(d,n)] = []
        for seed in range(5):
            results[(d,n)].append(experiment(d=d, n=n))
        
        print("d=",d, ", n=", n," results:", results[(d,n)])

d= 15 , n= 100  results: [0.23, 0.23, 0.14, 0.21, 0.21]
d= 15 , n= 300  results: [0.18, 0.28, 0.19, 0.23, 0.26]
d= 15 , n= 500  results: [0.41, 0.27, 0.3, 0.33, 0.38]
d= 15 , n= 700  results: [0.34, 0.35, 0.4, 0.37, 0.37]
d= 15 , n= 900  results: [0.42, 0.4, 0.33, 0.29, 0.38]


In [ ]:
# get confidence intervals for even these things, plot as fn of n, and multiple vals of d